# Logistic Regression with Subset Selection

## Variant 1: Feature Subset Selection for Heart Attack Prediction

**Goal**: Find the smallest useful set of predictors for heart attack prediction

**Methods**:
- Best subset selection
- Forward stepwise selection
- Backward stepwise selection

**Selection Criteria**: AIC/BIC optimization

**Validation**: 10-fold cross-validation for error/AUC confirmation

In [13]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Statistical modeling
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Machine learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

# Data loading
import kagglehub
import os

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


In [14]:
# Download and load the dataset
path = kagglehub.dataset_download("kamilpytlak/personal-key-indicators-of-heart-disease")
print("Path to dataset files:", path)

# Load the CSV file
csv_file = os.path.join(path, '2022', 'heart_2022_no_nans.csv')
df = pd.read_csv(csv_file)

print(f"Dataset shape: {df.shape}")
print(f"Target variable distribution:")
print(df['HadHeartAttack'].value_counts())

Path to dataset files: /Users/jackiewang/.cache/kagglehub/datasets/kamilpytlak/personal-key-indicators-of-heart-disease/versions/6
Dataset shape: (246022, 40)
Target variable distribution:
HadHeartAttack
No     232587
Yes     13435
Name: count, dtype: int64


In [15]:
# Define candidate features for selection
# Based on research questions: diabetes, stroke, general health + demographics + lifestyle

candidate_features = {
    'demographic': ['Sex', 'AgeCategory', 'RaceEthnicityCategory'],
    'health_conditions': ['HadDiabetes', 'HadStroke', 'GeneralHealth', 'HadAsthma', 
                          'HadKidneyDisease', 'HadCOPD', 'HadArthritis', 'HadDepressiveDisorder'],
    'lifestyle': ['SmokerStatus', 'AlcoholDrinkers', 'PhysicalActivities'],
    'health_metrics': ['BMI', 'PhysicalHealthDays', 'MentalHealthDays', 'SleepHours']
}

# Flatten all candidate features
all_features = []
for category, features in candidate_features.items():
    all_features.extend(features)

print(f"Total candidate features: {len(all_features)}")
print("\nFeatures by category:")
for category, features in candidate_features.items():
    print(f"  {category}: {features}")

# Create working dataset
df_model = df[all_features + ['HadHeartAttack']].copy()
print(f"\nWorking dataset shape: {df_model.shape}")
print(f"Missing values: {df_model.isnull().sum().sum()}")

Total candidate features: 18

Features by category:
  demographic: ['Sex', 'AgeCategory', 'RaceEthnicityCategory']
  health_conditions: ['HadDiabetes', 'HadStroke', 'GeneralHealth', 'HadAsthma', 'HadKidneyDisease', 'HadCOPD', 'HadArthritis', 'HadDepressiveDisorder']
  lifestyle: ['SmokerStatus', 'AlcoholDrinkers', 'PhysicalActivities']
  health_metrics: ['BMI', 'PhysicalHealthDays', 'MentalHealthDays', 'SleepHours']

Working dataset shape: (246022, 19)
Missing values: 0


In [16]:
# Data preprocessing for modeling
print("=== DATA PREPROCESSING ===")

# Encode categorical variables
categorical_cols = df_model.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('HadHeartAttack')  # Remove target variable

print(f"Categorical columns to encode: {categorical_cols}")

# Create dummy variables
df_encoded = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

# Convert target to binary
df_encoded['y'] = (df_encoded['HadHeartAttack'] == 'Yes').astype(int)
df_encoded = df_encoded.drop('HadHeartAttack', axis=1)

print(f"Encoded dataset shape: {df_encoded.shape}")
print(f"Number of features: {df_encoded.shape[1] - 1}")

# Split into features and target
X = df_encoded.drop('y', axis=1)
y = df_encoded['y']

print(f"\nFinal feature set: {X.shape[1]} features")
print(f"Target distribution: {y.value_counts().to_dict()}")

=== DATA PREPROCESSING ===
Categorical columns to encode: ['Sex', 'AgeCategory', 'RaceEthnicityCategory', 'HadDiabetes', 'HadStroke', 'GeneralHealth', 'HadAsthma', 'HadKidneyDisease', 'HadCOPD', 'HadArthritis', 'HadDepressiveDisorder', 'SmokerStatus', 'AlcoholDrinkers', 'PhysicalActivities']
Encoded dataset shape: (246022, 40)
Number of features: 39

Final feature set: 39 features
Target distribution: {0: 232587, 1: 13435}


In [17]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale numerical features
numerical_features = ['BMI', 'PhysicalHealthDays', 'MentalHealthDays', 'SleepHours']
scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Scale only numerical columns that exist in the dataset
num_cols_to_scale = [col for col in numerical_features if col in X_train.columns]
if num_cols_to_scale:
    X_train_scaled[num_cols_to_scale] = scaler.fit_transform(X_train[num_cols_to_scale])
    X_test_scaled[num_cols_to_scale] = scaler.transform(X_test[num_cols_to_scale])

print(f"Training set: {X_train_scaled.shape}")
print(f"Test set: {X_test_scaled.shape}")
print(f"Scaled features: {num_cols_to_scale}")

Training set: (196817, 39)
Test set: (49205, 39)
Scaled features: ['BMI', 'PhysicalHealthDays', 'MentalHealthDays', 'SleepHours']


## Method 1: Best Subset Selection

**Approach**: Evaluate all possible combinations of features up to a maximum size
**Criteria**: Select model with lowest AIC/BIC
**Computational Note**: Limited to reasonable subset sizes due to 2^p complexity

In [18]:
def fit_logistic_model(X_subset, y, feature_names=None):
    """Fit logistic regression and return AIC, BIC, and model info"""
    try:
        # Add constant
        X_with_const = sm.add_constant(X_subset)
        
        # Fit model
        model = Logit(y, X_with_const)
        result = model.fit(disp=False, maxiter=1000)
        
        return {
            'aic': result.aic,
            'bic': result.bic,
            'll': result.llf,
            'n_params': len(result.params),
            'converged': result.mle_retvals['converged'],
            'features': feature_names if feature_names else list(range(X_subset.shape[1])),
            'result': result
        }
    except Exception as e:
        return {
            'aic': np.inf,
            'bic': np.inf, 
            'll': -np.inf,
            'n_params': X_subset.shape[1] + 1,
            'converged': False,
            'features': feature_names if feature_names else list(range(X_subset.shape[1])),
            'result': None,
            'error': str(e)
        }

def best_subset_selection(X, y, max_features=10):
    """Perform best subset selection up to max_features"""
    feature_names = X.columns.tolist()
    n_features = len(feature_names)
    
    print(f"Starting best subset selection with {n_features} features")
    print(f"Evaluating subsets up to size {min(max_features, n_features)}")
    
    best_models = {}
    
    # Try subsets of increasing size
    for k in range(1, min(max_features, n_features) + 1):
        print(f"\nEvaluating subsets of size {k}...")
        
        best_aic = np.inf
        best_bic = np.inf
        best_model_aic = None
        best_model_bic = None
        
        # Generate all combinations of k features
        n_combinations = len(list(combinations(range(n_features), k)))
        print(f"  Testing {n_combinations} combinations")
        
        for i, feature_indices in enumerate(combinations(range(n_features), k)):
            if i > 0 and i % 1000 == 0:
                print(f"    Processed {i}/{n_combinations} combinations")
            
            selected_features = [feature_names[j] for j in feature_indices]
            X_subset = X[selected_features]
            
            model_info = fit_logistic_model(X_subset, y, selected_features)
            
            # Track best AIC model
            if model_info['aic'] < best_aic:
                best_aic = model_info['aic']
                best_model_aic = model_info.copy()
            
            # Track best BIC model
            if model_info['bic'] < best_bic:
                best_bic = model_info['bic']
                best_model_bic = model_info.copy()
        
        best_models[k] = {
            'aic_model': best_model_aic,
            'bic_model': best_model_bic
        }
        
        print(f"  Best AIC ({k} features): {best_aic:.3f}, Features: {best_model_aic['features']}")
        print(f"  Best BIC ({k} features): {best_bic:.3f}, Features: {best_model_bic['features']}")
    
    return best_models

# Run best subset selection (limit to 8 features for computational feasibility)
print("=== BEST SUBSET SELECTION ===")
best_models = best_subset_selection(X_train_scaled, y_train, max_features=8)

=== BEST SUBSET SELECTION ===
Starting best subset selection with 39 features
Evaluating subsets up to size 8

Evaluating subsets of size 1...
  Testing 39 combinations
  Best AIC (1 features): 80757.463, Features: ['PhysicalHealthDays']
  Best BIC (1 features): 80777.843, Features: ['PhysicalHealthDays']

Evaluating subsets of size 2...
  Testing 741 combinations
  Best AIC (2 features): 80718.389, Features: ['BMI', 'PhysicalHealthDays']
  Best BIC (2 features): 80748.959, Features: ['BMI', 'PhysicalHealthDays']

Evaluating subsets of size 3...
  Testing 9139 combinations
    Processed 1000/9139 combinations
    Processed 2000/9139 combinations
    Processed 3000/9139 combinations
    Processed 4000/9139 combinations
    Processed 5000/9139 combinations
    Processed 6000/9139 combinations
    Processed 7000/9139 combinations
    Processed 8000/9139 combinations
    Processed 9000/9139 combinations
  Best AIC (3 features): 80681.352, Features: ['BMI', 'PhysicalHealthDays', 'MentalHeal

KeyboardInterrupt: 

## Method 2: Forward Stepwise Selection

**Approach**: Start with no variables, add one at a time
**Selection**: At each step, add the variable that most improves AIC/BIC
**Stopping**: When no variable improves the criterion

In [ ]:
def forward_selection(X, y, criterion='aic', max_features=None):
    """Forward stepwise selection based on AIC or BIC"""
    feature_names = X.columns.tolist()
    n_features = len(feature_names)
    max_features = max_features or n_features
    
    selected_features = []
    remaining_features = feature_names.copy()
    
    selection_history = []
    
    print(f"\n=== FORWARD SELECTION (criterion: {criterion.upper()}) ===")
    
    # Start with intercept-only model
    baseline_model = fit_logistic_model(pd.DataFrame(np.ones(len(y))), y, ['intercept_only'])
    current_score = baseline_model[criterion]
    
    print(f"Baseline (intercept-only) {criterion.upper()}: {current_score:.3f}")
    
    step = 0
    while remaining_features and len(selected_features) < max_features:
        step += 1
        print(f"\nStep {step}: Testing {len(remaining_features)} candidates")
        
        best_score = current_score
        best_feature = None
        
        # Try adding each remaining feature
        for feature in remaining_features:
            test_features = selected_features + [feature]
            X_subset = X[test_features]
            
            model_info = fit_logistic_model(X_subset, y, test_features)
            score = model_info[criterion]
            
            if score < best_score:  # Lower AIC/BIC is better
                best_score = score
                best_feature = feature
        
        # Check if we found an improvement
        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            current_score = best_score
            
            print(f"  Added: {best_feature}")
            print(f"  New {criterion.upper()}: {current_score:.3f}")
            print(f"  Selected features: {selected_features}")
            
            selection_history.append({
                'step': step,
                'added_feature': best_feature,
                'features': selected_features.copy(),
                'score': current_score
            })
        else:
            print(f"  No improvement found. Stopping.")
            break
    
    # Fit final model
    if selected_features:
        final_model = fit_logistic_model(X[selected_features], y, selected_features)
    else:
        final_model = baseline_model
    
    return {
        'selected_features': selected_features,
        'final_model': final_model,
        'selection_history': selection_history,
        'criterion': criterion
    }

# Run forward selection with both AIC and BIC
forward_aic = forward_selection(X_train_scaled, y_train, criterion='aic', max_features=15)
forward_bic = forward_selection(X_train_scaled, y_train, criterion='bic', max_features=15)

## Method 3: Backward Stepwise Selection

**Approach**: Start with all variables, remove one at a time
**Selection**: At each step, remove the variable that most improves AIC/BIC
**Stopping**: When removing any variable worsens the criterion

In [ ]:
def backward_elimination(X, y, criterion='aic', min_features=1):
    """Backward elimination based on AIC or BIC"""
    feature_names = X.columns.tolist()
    selected_features = feature_names.copy()
    
    elimination_history = []
    
    print(f"\n=== BACKWARD ELIMINATION (criterion: {criterion.upper()}) ===")
    
    # Start with full model
    current_model = fit_logistic_model(X[selected_features], y, selected_features)
    current_score = current_model[criterion]
    
    print(f"Starting with {len(selected_features)} features")
    print(f"Initial {criterion.upper()}: {current_score:.3f}")
    
    step = 0
    while len(selected_features) > min_features:
        step += 1
        print(f"\nStep {step}: Testing removal of {len(selected_features)} features")
        
        best_score = current_score
        worst_feature = None
        
        # Try removing each feature
        for feature in selected_features:
            test_features = [f for f in selected_features if f != feature]
            if len(test_features) == 0:  # Don't remove last feature
                continue
                
            X_subset = X[test_features]
            model_info = fit_logistic_model(X_subset, y, test_features)
            score = model_info[criterion]
            
            if score < best_score:  # Lower AIC/BIC is better
                best_score = score
                worst_feature = feature
        
        # Check if we found an improvement
        if worst_feature is not None:
            selected_features.remove(worst_feature)
            current_score = best_score
            
            print(f"  Removed: {worst_feature}")
            print(f"  New {criterion.upper()}: {current_score:.3f}")
            print(f"  Remaining features ({len(selected_features)}): {selected_features}")
            
            elimination_history.append({
                'step': step,
                'removed_feature': worst_feature,
                'features': selected_features.copy(),
                'score': current_score
            })
        else:
            print(f"  No improvement found. Stopping.")
            break
    
    # Fit final model
    final_model = fit_logistic_model(X[selected_features], y, selected_features)
    
    return {
        'selected_features': selected_features,
        'final_model': final_model,
        'elimination_history': elimination_history,
        'criterion': criterion
    }

# Run backward elimination with both AIC and BIC
backward_aic = backward_elimination(X_train_scaled, y_train, criterion='aic', min_features=3)
backward_bic = backward_elimination(X_train_scaled, y_train, criterion='bic', min_features=3)

## Model Comparison and Selection

**Strategy**: Compare all methods and select the winner based on:
1. **Statistical Criteria**: AIC and BIC scores
2. **Validation Performance**: 10-fold cross-validation AUC
3. **Interpretability**: Number of features and practical significance

In [ ]:
# Collect all candidate models
candidate_models = []

# Add best subset models
print("=== COLLECTING CANDIDATE MODELS ===")
for k, models in best_models.items():
    # AIC winner for this subset size
    if models['aic_model']['converged']:
        candidate_models.append({
            'method': f'Best Subset (k={k})',
            'criterion_used': 'AIC',
            'features': models['aic_model']['features'],
            'n_features': len(models['aic_model']['features']),
            'aic': models['aic_model']['aic'],
            'bic': models['aic_model']['bic'],
            'model_info': models['aic_model']
        })
    
    # BIC winner for this subset size
    if models['bic_model']['converged']:
        candidate_models.append({
            'method': f'Best Subset (k={k})',
            'criterion_used': 'BIC',
            'features': models['bic_model']['features'],
            'n_features': len(models['bic_model']['features']),
            'aic': models['bic_model']['aic'],
            'bic': models['bic_model']['bic'],
            'model_info': models['bic_model']
        })

# Add forward selection models
if forward_aic['final_model']['converged']:
    candidate_models.append({
        'method': 'Forward Selection',
        'criterion_used': 'AIC',
        'features': forward_aic['selected_features'],
        'n_features': len(forward_aic['selected_features']),
        'aic': forward_aic['final_model']['aic'],
        'bic': forward_aic['final_model']['bic'],
        'model_info': forward_aic['final_model']
    })

if forward_bic['final_model']['converged']:
    candidate_models.append({
        'method': 'Forward Selection',
        'criterion_used': 'BIC',
        'features': forward_bic['selected_features'],
        'n_features': len(forward_bic['selected_features']),
        'aic': forward_bic['final_model']['aic'],
        'bic': forward_bic['final_model']['bic'],
        'model_info': forward_bic['final_model']
    })

# Add backward elimination models
if backward_aic['final_model']['converged']:
    candidate_models.append({
        'method': 'Backward Elimination',
        'criterion_used': 'AIC',
        'features': backward_aic['selected_features'],
        'n_features': len(backward_aic['selected_features']),
        'aic': backward_aic['final_model']['aic'],
        'bic': backward_aic['final_model']['bic'],
        'model_info': backward_aic['final_model']
    })

if backward_bic['final_model']['converged']:
    candidate_models.append({
        'method': 'Backward Elimination',
        'criterion_used': 'BIC',
        'features': backward_bic['selected_features'],
        'n_features': len(backward_bic['selected_features']),
        'aic': backward_bic['final_model']['aic'],
        'bic': backward_bic['final_model']['bic'],
        'model_info': backward_bic['final_model']
    })

print(f"Collected {len(candidate_models)} candidate models")

# Create comparison dataframe
comparison_df = pd.DataFrame([{
    'Method': model['method'],
    'Criterion': model['criterion_used'],
    'N_Features': model['n_features'],
    'AIC': model['aic'],
    'BIC': model['bic'],
    'Features': ', '.join(model['features'][:3]) + ('...' if len(model['features']) > 3 else '')
} for model in candidate_models])

# Sort by AIC
comparison_df = comparison_df.sort_values('AIC').reset_index(drop=True)
print("\n=== MODEL COMPARISON (sorted by AIC) ===")
print(comparison_df.to_string(index=False, float_format='%.3f'))

In [ ]:
def cross_validate_model(features, X, y, cv_folds=10):
    """Perform cross-validation for a feature set"""
    from sklearn.model_selection import cross_val_score, StratifiedKFold
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score, make_scorer
    
    # Prepare data
    X_subset = X[features]
    
    # Use sklearn for reliable cross-validation
    lr = LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')
    
    # Stratified K-fold to maintain class balance
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    # Calculate AUC scores
    auc_scores = cross_val_score(lr, X_subset, y, cv=cv, scoring='roc_auc')
    
    return {
        'mean_auc': auc_scores.mean(),
        'std_auc': auc_scores.std(),
        'auc_scores': auc_scores
    }

# Cross-validate top models (by AIC)
print("\n=== CROSS-VALIDATION OF TOP MODELS ===")

# Select top 5 models by AIC for CV
top_models = candidate_models[:5]

cv_results = []
for i, model in enumerate(top_models):
    print(f"\nCV for Model {i+1}: {model['method']} ({model['criterion_used']})")
    print(f"Features ({model['n_features']}): {model['features']}")
    
    cv_result = cross_validate_model(model['features'], X_train_scaled, y_train)
    
    cv_results.append({
        'model_index': i,
        'method': model['method'],
        'criterion': model['criterion_used'],
        'n_features': model['n_features'],
        'aic': model['aic'],
        'bic': model['bic'],
        'cv_auc_mean': cv_result['mean_auc'],
        'cv_auc_std': cv_result['std_auc'],
        'features': model['features']
    })
    
    print(f"CV AUC: {cv_result['mean_auc']:.4f} (±{cv_result['std_auc']:.4f})")

# Create final comparison
final_comparison = pd.DataFrame(cv_results)
print("\n=== FINAL MODEL COMPARISON ===")
print(final_comparison[['method', 'criterion', 'n_features', 'aic', 'bic', 'cv_auc_mean', 'cv_auc_std']].to_string(index=False, float_format='%.4f'))

## Winner Selection and Final Model

**Selection Criteria**:
1. **Primary**: Lowest AIC (penalizes complexity less than BIC)
2. **Validation**: Competitive cross-validation AUC
3. **Practical**: Reasonable number of features for interpretation

**Final Model**: The winner will be our minimal useful predictor set

In [ ]:
# Select the winner
winner_idx = final_comparison['aic'].idxmin()
winner = final_comparison.loc[winner_idx]

print("=== WINNER SELECTION ===")
print(f"Method: {winner['method']} ({winner['criterion']})")
print(f"Number of features: {winner['n_features']}")
print(f"AIC: {winner['aic']:.3f}")
print(f"BIC: {winner['bic']:.3f}")
print(f"Cross-validation AUC: {winner['cv_auc_mean']:.4f} (±{winner['cv_auc_std']:.4f})")
print(f"\nSelected features: {winner['features']}")

# Fit final model on full training set
final_features = winner['features']
X_final = X_train_scaled[final_features]
X_final_with_const = sm.add_constant(X_final)

final_model = Logit(y_train, X_final_with_const).fit()

print("\n=== FINAL MODEL SUMMARY ===")
print(final_model.summary())

# Test set performance
X_test_final = X_test_scaled[final_features]
X_test_final_with_const = sm.add_constant(X_test_final)

# Predictions
test_probs = final_model.predict(X_test_final_with_const)
test_preds = (test_probs > 0.5).astype(int)

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

test_accuracy = accuracy_score(y_test, test_preds)
test_precision = precision_score(y_test, test_preds)
test_recall = recall_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds)
test_auc = roc_auc_score(y_test, test_probs)

print("\n=== TEST SET PERFORMANCE ===")
print(f"Accuracy:  {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1-Score:  {test_f1:.4f}")
print(f"AUC:       {test_auc:.4f}")

# Feature importance (coefficients)
feature_importance = pd.DataFrame({
    'feature': final_features,
    'coefficient': final_model.params[1:],  # Exclude intercept
    'p_value': final_model.pvalues[1:],
    'conf_low': final_model.conf_int()[0][1:],
    'conf_high': final_model.conf_int()[1][1:]
})
feature_importance['odds_ratio'] = np.exp(feature_importance['coefficient'])
feature_importance = feature_importance.sort_values('coefficient', key=abs, ascending=False)

print("\n=== FEATURE IMPORTANCE (Minimal Useful Predictor Set) ===")
print(feature_importance.to_string(index=False, float_format='%.4f'))

In [ ]:
# Visualization of results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Model comparison (AIC vs BIC)
ax1 = axes[0, 0]
scatter = ax1.scatter(final_comparison['aic'], final_comparison['bic'], 
                     c=final_comparison['cv_auc_mean'], cmap='viridis', s=100)
ax1.set_xlabel('AIC')
ax1.set_ylabel('BIC')
ax1.set_title('Model Selection: AIC vs BIC\n(Color = CV AUC)')
plt.colorbar(scatter, ax=ax1, label='CV AUC')

# Mark winner
winner_aic = winner['aic']
winner_bic = winner['bic']
ax1.scatter(winner_aic, winner_bic, color='red', s=200, marker='*', 
           edgecolor='black', linewidth=2, label='Winner')
ax1.legend()

# 2. Feature importance
ax2 = axes[0, 1]
y_pos = np.arange(len(feature_importance))
bars = ax2.barh(y_pos, feature_importance['coefficient'], 
                color=['red' if x < 0 else 'blue' for x in feature_importance['coefficient']])
ax2.set_yticks(y_pos)
ax2.set_yticklabels(feature_importance['feature'])
ax2.set_xlabel('Coefficient')
ax2.set_title('Feature Coefficients (Minimal Set)')
ax2.axvline(x=0, color='black', linestyle='-', alpha=0.3)

# 3. ROC Curve
from sklearn.metrics import roc_curve
ax3 = axes[1, 0]
fpr, tpr, _ = roc_curve(y_test, test_probs)
ax3.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {test_auc:.3f})')
ax3.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random')
ax3.set_xlim([0.0, 1.0])
ax3.set_ylim([0.0, 1.05])
ax3.set_xlabel('False Positive Rate')
ax3.set_ylabel('True Positive Rate')
ax3.set_title('ROC Curve - Final Model')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Confusion Matrix
from sklearn.metrics import confusion_matrix
ax4 = axes[1, 1]
cm = confusion_matrix(y_test, test_preds)
im = ax4.imshow(cm, interpolation='nearest', cmap='Blues')
ax4.set_title('Confusion Matrix')
ax4.set_xlabel('Predicted Label')
ax4.set_ylabel('True Label')

# Add text annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax4.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("SUBSET SELECTION COMPLETE!")
print("="*60)
print(f"Best method: {winner['method']} using {winner['criterion']}")
print(f"Minimal useful predictor set: {len(final_features)} features")
print(f"Features: {final_features}")
print(f"Test AUC: {test_auc:.4f}")
print("="*60)